### O Net database set up

Setting up the database to easily query the O Net datasets

1. Download mysql script from O Net: https://www.onetcenter.org/database.html#all-files
2. Convert mysql .sql scripts to a combined sqlite script
3. Create database either pythonically or using command prmpt as below



In [8]:
import re
import os

def convert_mysql_to_sqlite(mysql_files_dir, sqlite_file_path):
    # Create or open the SQLite database
    with open(sqlite_file_path, 'w') as sqlite_file:
        print(f"opening {len(os.listdir(mysql_files_dir))} files")
        # Loop through each .sql file in the directory
        for filename in os.listdir(mysql_files_dir):
            if filename.endswith(".sql"):
                # Full path to the MySQL .sql file
                mysql_file_path = os.path.join(mysql_files_dir, filename)

                # Read the content of the MySQL SQL file
                with open(mysql_file_path, 'r') as mysql_file:
                    sql_content = mysql_file.read()

                    # Make necessary replacements to convert MySQL to SQLite syntax
                    sql_content = re.sub(r'AUTO_INCREMENT', 'INTEGER PRIMARY KEY AUTOINCREMENT', sql_content)
                    sql_content = re.sub(r'\bINT\(\d+\)\b', 'INTEGER', sql_content)
                    sql_content = re.sub(r'\bVARCHAR\(\d+\)\b', 'TEXT', sql_content)
                    sql_content = re.sub(r'\bDATETIME\b', 'TEXT', sql_content)
                    sql_content = re.sub(r'\bTIMESTAMP\b', 'TEXT', sql_content)
                    sql_content = re.sub(r'`', '', sql_content)  # Remove backticks for SQLite

                    # Remove MySQL-specific settings (like ENGINE and CHARSET)
                    sql_content = re.sub(r'ENGINE=\w+', '', sql_content)
                    sql_content = re.sub(r'CHARSET=\w+', '', sql_content)

                    # Write the converted content into the SQLite .sql file
                    sqlite_file.write(sql_content + "\n\n")

                print(f"Converted {filename} to SQLite-compatible format.")

    print(f"Converted SQL files saved to {sqlite_file_path}")

# Example usage:
mysql_files_directory = '../data/O_Net_MySQL/db_29_2_mysql/'  # Folder containing MySQL .sql files
sqlite_output_file = '../data/O_Net_SqlLite/o_net_sqlite_schema.sql'  # SQLite-compatible .sql file

# Convert the MySQL .sql files to a single SQLite .sql file
convert_mysql_to_sqlite(mysql_files_directory, sqlite_output_file)


opening 41 files
Converted 36_skills_to_work_context.sql to SQLite-compatible format.
Converted 24_dwa_reference.sql to SQLite-compatible format.
Converted 31_technology_skills.sql to SQLite-compatible format.
Converted 23_iwa_reference.sql to SQLite-compatible format.
Converted 28_unspsc_reference.sql to SQLite-compatible format.
Converted 17_task_statements.sql to SQLite-compatible format.
Converted 38_basic_interests_to_riasec.sql to SQLite-compatible format.
Converted 16_skills.sql to SQLite-compatible format.
Converted 13_interests.sql to SQLite-compatible format.
Converted 33_abilities_to_work_activities.sql to SQLite-compatible format.
Converted 14_job_zones.sql to SQLite-compatible format.
Converted 39_interests_illus_activities.sql to SQLite-compatible format.
Converted 06_level_scale_anchors.sql to SQLite-compatible format.
Converted 27_related_occupations.sql to SQLite-compatible format.
Converted 02_job_zone_reference.sql to SQLite-compatible format.
Converted 40_interests_

In [5]:
# import sqlite3

# def create_database_from_sqlite_sql(sqlite_file, sql_file):
#     # Connect to SQLite database (creates the database file if it doesn't exist)
#     conn = sqlite3.connect(sqlite_file)
#     cursor = conn.cursor()

#     # Open and read the converted SQL file
#     with open(sql_file, 'r') as file:
#         sql_content = file.read()

#     # Execute the SQL commands to create the tables and insert data
#     cursor.executescript(sql_content)

#     # Commit and close the connection
#     conn.commit()
#     conn.close()
#     print(f"Database {sqlite_file} created and data imported successfully!")

# # Example usage:
# sqlite_file = '../data/O_Net_SqlLite/onet_database.db'  # SQLite database file
# sql_file = '../data/O_Net_SqlLite/o_net_sqlite_schema.sql'  # Converted SQL file

# create_database_from_sqlite_sql(sqlite_file, sql_file)


### Alternate option

In [ ]:
!sqlite3 path/to/your/onet_database.db
!.read path/to/your/o_net_sqlite_schema.sql

### Annotations Tables

Following the research work "A novel approach for job matching and skill recommendation using transformers and the O*NET database", we are creating tables for the annotations in the o_net database.

Paper: https://www.sciencedirect.com/science/article/pii/S2214579625000048
Github: https://gitlab.com/hri_lab1/using-transformers-and-o-net-to-match-jobs-to-applicants-resumes

1. Create tables
2. Insert data from the json files

In [9]:
!sqlite3 ../data/annotations_scenario_1/annotations_scenario_1.db < d../ata/annotations_scenario_1/annotations_scenario_1.sql


In [2]:
import sqlite3
import json
import os

# SQLite database file
db_file = "../data/annotations_scenario_1/annotations_scenario_1.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Directory containing JSON files for Scenario 1
scenario_1_dir = "../data/annotations_scenario_1"

# Function to insert JSON data
def insert_annotations(json_file):
    try:
        with open(json_file, "r", encoding="utf-8") as file:
            resume_data = json.load(file)

        for resume in resume_data:
            # Insert into resumes table
            cursor.execute(
                "INSERT INTO resumes (resume_text, original_job) VALUES (?, ?)",
                (resume.get("resume_text", ""), resume.get("original_job", "Unknown")),
            )
            resume_id = cursor.lastrowid  # Get the inserted resume ID

            # Insert predicted jobs (from the research model)
            for job in resume.get("predicted_jobs", []):
                cursor.execute(
                    "INSERT INTO predicted_jobs (resume_id, job_code, job_title) VALUES (?, ?, ?)",
                    (resume_id, job[0], job[1]),
                )

            # Insert annotations (human expert ratings) safely
            for annotator_id, key in enumerate(["annotation_1", "annotation_2", "annotation_3"], start=1):
                rating = resume.get(key)  # Use .get() to avoid KeyError
                if rating is not None:  # Insert only if the annotation exists
                    cursor.execute(
                        "INSERT INTO annotations (resume_id, annotator_id, rating) VALUES (?, ?, ?)",
                        (resume_id, annotator_id, rating),
                    )

            # Insert naive algorithm predictions
            for key, algo in zip(["naive_alg1_jobs", "naive_alg2_jobs"], ["naive_alg1", "naive_alg2"]):
                for job in resume.get(key, []):  # Use .get() to avoid KeyError
                    cursor.execute(
                        "INSERT INTO naive_algorithm_predictions (resume_id, algorithm_version, job_title, similarity_score) VALUES (?, ?, ?, ?)",
                        (resume_id, algo, job[0], job[1]),
                    )

        print(f"✅ Processed: {os.path.basename(json_file)}")  # Show progress

    except Exception as e:
        print(f"❌ Error processing {json_file}: {e}")

# Scan directory and process only JSON files
json_files = [f for f in os.listdir(scenario_1_dir) if f.endswith(".json")]

if not json_files:
    print("⚠️ No JSON files found in the directory!")
else:
    print(f"📂 Found {len(json_files)} JSON files. Processing...")
    for json_file in json_files:
        insert_annotations(os.path.join(scenario_1_dir, json_file))

# Commit and close
conn.commit()
conn.close()

print("🎉 Scenario 1 data inserted successfully!")


📂 Found 3 JSON files. Processing...
✅ Processed: naive_alg2_results - annotator_1.json
✅ Processed: naive_alg2_results - annotator_3.json
✅ Processed: naive_alg2_results - annotator_2.json
🎉 Scenario 1 data inserted successfully!
